In [34]:
# Purpose:  Model topics for group 5 ( 9th republic debate and 6th dem debate)
# Author:  Preeti Javaji
# Date:  05/17/16
# Course:  MAS DSE capstone, Spring 2016

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import math

pd.set_option('display.max_colwidth', 240)
pd.set_option('display.max_columns', 50)

import sys
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB


print 'done'

done


In [36]:
# Function defs.

# return string with non-ascii chars removed
def to_ascii(s):
    returnstr = s.strip()
    returnstr = "".join([ch for ch in returnstr if ord(ch)< 128])
    return returnstr

print 'done'

done


In [38]:
# Load data file & clean the 'text' column.

df = pd.read_csv("Sentiment_backup.csv")

# Convert to ascii
df['ascii'] = df['text'].apply(to_ascii)
#df['ascii'] = df['ascii'].str.encode('ascii',errors='ignore') 
df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
df['ascii'] = df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
df['ascii'] = df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
df['ascii'] = df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
df['ascii'] = df['ascii'].str.lower()

df['K_sentiment'] = df['sentiment']
print df.shape
#print df[df['ascii']!=df['text']][['ascii', 'K_sentiment']].head(10)
df[['text', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)

(4333, 23)


,text,ascii
0,RT @TheBaxterBean: Scott Walker's Abortion Ban Allows Rapist Father To Sue For Emotional Distress http://t.co/rHMvgumuir #GOPDebate @Badger��_,rt @thebaxterbean scott walkers abortion ban allows rapist father to sue for emotional distress url #gopdebate @badger_
1,"RT @mydaughtersarmy: Dear GOP,\rIf you want to stop abortions... \r\r#GOPDebate \rhttp://t.co/RbL1t4iPx6","rt @mydaughtersarmy dear gop,\rif you want to stop abortions... \r\r#gopdebate \rurl"
2,RT @Bipartisanism: A message to the #GOPDebate candidates: http://t.co/WFK5CmVgiN,rt @bipartisanism a message to the #gopdebate candidates url


In [39]:
# Show the class frequencies.
counts = df['subject_matter'].value_counts()
total = counts.sum()

counts = counts.apply(lambda x: ("Count: %d" % x, "Frequency: %.2f" % (float(x)/total)) )

#print("%.2f" % a)
#string = 'string%d' % (i,)

print counts
print type(counts)

None of the above                       (Count: 1271, Frequency: 0.33)
Religion                                 (Count: 407, Frequency: 0.11)
Women's Issues (not abortion though)     (Count: 362, Frequency: 0.10)
Racial issues                            (Count: 353, Frequency: 0.09)
Abortion                                 (Count: 293, Frequency: 0.08)
Jobs and Economy                         (Count: 251, Frequency: 0.07)
Immigration                              (Count: 211, Frequency: 0.06)
FOX News or Moderators                   (Count: 188, Frequency: 0.05)
LGBT issues                              (Count: 126, Frequency: 0.03)
iran deal                                 (Count: 74, Frequency: 0.02)
Healthcare (including Medicare)           (Count: 67, Frequency: 0.02)
Gun Control                               (Count: 61, Frequency: 0.02)
Foreign Policy                            (Count: 41, Frequency: 0.01)
ISIS                                      (Count: 40, Frequency: 0.01)
none  

In [40]:
%pwd

u'/Users/preetijavaji/Documents/DSE/capstone'

In [41]:
# Add the 1st democratic debate transcript
D1_df = pd.read_csv("labeled_transcripts/R9_proc_labelled.csv")
D1_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,topic
0,0,50,APPLAUSE,audience,575,[applause],NaN
1,1,50,APPLAUSE,audience,691,[applause],NaN
2,2,50,APPLAUSE,audience,729,[applause],NaN
3,3,50,APPLAUSE,audience,756,[applause],NaN
4,4,50,APPLAUSE,audience,793,[applause],NaN


In [42]:
#clean it up too
D1_df['ascii'] = D1_df['text'].apply(to_ascii)

D1_df['ascii'] = D1_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D1_df['ascii'] = D1_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D1_df['ascii'] = D1_df['ascii'].str.lower()

In [43]:
# Now take the text and topic from both the dataframes and concat
D1_new = D1_df[['ascii','topic']]
T_DF = df[['ascii','subject_matter']]

T_DF = T_DF.rename(columns = {'subject_matter':'topic'})

In [44]:
# read the next debate transcript
D2_df = pd.read_csv("labeled_transcripts/D6_proc_labeled.csv")
D2_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,topic
0,0,543,WOODRUFF,moderator,0,"WOODRUFF: Good evening, and thank you. We are happy to welcome you to Milwaukee for this Democratic debate. We are especially pleased to thank our partners at Facebook, who have helped us set up a vibrant conversation among voters who a...",intro
1,1,50,APPLAUSE,audience,1153,[applause],intro
2,2,610,IFILL,moderator,543,"IFILL: We want to also extend our warm thanks to Milwaukee Public Radio and Milwaukee Public Television, as well as all of our friends at the PBS member stations across the country tuning in tonight. This is the sixth time the Democrats...",intro
3,3,50,APPLAUSE,audience,1246,[applause],intro
4,4,124,WOODRUFF,moderator,1153,"WOODRUFF: Welcome, Senator, great to see you. And former Secretary of State Hillary Clinton. Very good to be here with you.",intro


In [45]:
#clean it up too
D2_df['ascii'] = D2_df['text'].apply(to_ascii)

D2_df['ascii'] = D2_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D2_df['ascii'] = D2_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D2_df['ascii'] = D2_df['ascii'].str.lower()

In [46]:
# Now take the text and topic from both the dataframes and concat
D2_new = D2_df[['ascii','topic']]

In [52]:
# read stuff from hashtags .csv
D3_df = pd.read_csv("labeled_transcripts/hashtagTweets_updated_1.csv")
D3_df.tail()

,topic,tweetid,text
1726,supremecourt,7.067650e+17,"""Possible #SupremeCourt Pick Championed Black History Museum #President #BarackObama #RobertL.Wilkins... https://t.co/kZwQyiHh1M """
1727,supremecourt,7.067620e+17,"""Tell @tedcruz: stop obstructing @POTUS #SCOTUS nominee. #doyourjob. Sign the petition https://t.co/Fw5l3eCs3l via @CREDOMobile #SupremeCourt """
1728,supremecourt,7.066720e+17,"""Obama has the right to nominate the next #SupremeCourt Justice. #DemDebate #ImWithHer #HillaryClinton """
1729,supremecourt,7.066670e+17,"""Joe #Biden jokingly suggests #Obama should nominate Ted #Cruz to the #SupremeCourt https://t.co/YoMYoHz0vY via @TheWeek Typical Biden _���_��� """
1730,supremecourt,7.066190e+17,"""#Biden To #Obama: ���If You Want To Remake The #SupremeCourt, Here�۪s What You Can Do��_�۪ https://t.co/xNXFMMWPPa - #POTUS #SCOTUS #DoYourJob """


In [53]:
#clean it up too
D3_df['ascii'] = D3_df['text'].apply(to_ascii)

D3_df['ascii'] = D3_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D3_df['ascii'] = D3_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D3_df['ascii'] = D3_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D3_df['ascii'] = D3_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D3_df['ascii'] = D3_df['ascii'].str.lower()

In [54]:
# Now concat all 4
D1_new = D1_df[['ascii','topic']]
T_DF = df[['ascii','subject_matter']]

T_DF = T_DF.rename(columns = {'subject_matter':'topic'})
D2_new = D2_df[['ascii','topic']]
T_new1 = pd.concat([T_DF,D1_new])
print T_new1.shape
D3_new = D3_df[['ascii','topic']]
T_new2 = pd.concat([T_new1,D2_new])
print T_new1.shape
T_new = pd.concat([T_new2,D3_new])
print T_new.shape

(4772, 2)
(4772, 2)
(6714, 2)


In [55]:
# clean the topics before encoding
# first convert to lower case
T_new['topic'] = T_new['topic'].str.lower()
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 'abortion',
 'benghazi',
 'billclinton',
 'black incarceration',
 'border security',
 'bordersecurity',
 'campaign finance',
 'clinton emails',
 'closing',
 'cnn',
 'cnndebate',
 'cnntownhall',
 'college',
 'college tuition',
 'common core',
 'conservatism',
 'deathpenalty',
 'economy',
 'environment',
 'family leave',
 'familyleave',
 'flintwater',
 'flintwatercrisis',
 'foreign aid',
 'foreign policy',
 'fox news or moderators',
 'ground troops',
 'gun control',
 'healthcare',
 'healthcare (including medicare)',
 'homeland security',
 'homelandsecurity',
 'immigration',
 'immigration reform',
 'intro',
 'iran',
 'iran deal',
 'iraq',
 'isis',
 'israel',
 'jobs and economy',
 'lgbt issues',
 'marijuana',
 'medicaid',
 'mortage crisis',
 'national security',
 'none',
 'none of the above',
 'obama',
 'obamacare',
 'poverty',
 'president bush',
 'presidential perch',
 'race relations',
 'racial issues',
 'radical islam',
 'regime change',
 'religion',
 'role of government',
 'rona

# After seeing the list above we will combine:
1. 'border security','bordersecurity', -- group with immigration
2. 'cnn','cnndebate','cnntownhall',
3. 'college','college tuition',
4. 'family leave','familyleave',
5. 'flintwater','flintwatercrisis',
6. 'foreign aid','foreign policy', 'ground troops', 'homeland security','homelandsecurity', 'national security', 'russia',
7. 'fox news or moderators',--- not needed
8. 'healthcare', 'obamacare','healthcare (including medicare)','medicaid',
9. 'immigration','immigration reform',
10. 'iran','iran deal',
11.'isis','radical islam',
12.  'jobs and economy',  'economy',
13. 'none','none of the above',
14. 'racial issues', 'black incarceration', 'race relations',
15. 'syria','syrian refugees',
17. 'wall street','taxes', 'mortage crisis',
18. "women's issues (not abortion though)",'womens vote',
19. 'zika','zikavirus'

In [61]:
T_new.ix[T_new['topic'].isin( ['cnndebate','cnntownhall']),'topic'] = 'cnn' #2
T_new.ix[T_new['topic'] == 'college tuition','topic'] = 'college' #3
T_new.ix[T_new['topic'] == 'familyleave','topic'] = 'family leave'#4
T_new.ix[T_new['topic'] == 'flintwatercrisis','topic'] = 'flintwater' #5
T_new.ix[T_new['topic'].isin( ['foreign aid','ground troops', 'homeland security','homelandsecurity', 'national security', 'russia']),
         'topic'] = 'foreign policy' #6
T_new.ix[T_new['topic'].isin( ['healthcare (including medicare)', 'medicaid','obamacare']),'topic'] = 'healthcare' #8
T_new.ix[T_new['topic'].isin( ['border security','bordersecurity','immigration reform']),'topic'] = 'immigration' #9

T_new.ix[T_new['topic'] == 'iran deal','topic'] = 'iran'  #10
T_new.ix[T_new['topic'] == 'radical islam','topic'] = 'isis' #11
T_new.ix[T_new['topic'] == 'economy','topic'] = 'jobs and economy' #12
T_new.ix[T_new['topic'] == 'none of the above','topic'] = 'none' #13
T_new.ix[T_new['topic'].isin( ['black incarceration', 'race relations']),'topic'] = 'racial issues' #14
T_new.ix[T_new['topic'] == 'syrian refugees','topic'] = 'syria' #15
T_new.ix[T_new['topic'].isin( ['taxes', 'mortage crisis']),'topic'] = 'wall street' #17
T_new.ix[T_new['topic'] == 'womens vote','topic'] =  "women's issues (not abortion though)" #18
T_new.ix[T_new['topic'] == 'zika','topic'] = 'zikavirus'   #19
T_new.ix[T_new['topic'] == 'supreme courth nominee','topic'] = 'supremecourt'


In [62]:
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

['abortion',
 'benghazi',
 'billclinton',
 'campaign finance',
 'clinton emails',
 'cnn',
 'college',
 'common core',
 'conservatism',
 'deathpenalty',
 'environment',
 'family leave',
 'flintwater',
 'foreign policy',
 'fox news or moderators',
 'gun control',
 'healthcare',
 'immigration',
 'iran',
 'iraq',
 'isis',
 'israel',
 'jobs and economy',
 'lgbt issues',
 'marijuana',
 'none',
 'obama',
 'poverty',
 'president bush',
 'presidential perch',
 'racial issues',
 'regime change',
 'religion',
 'role of government',
 'ronald reagan',
 'ronaldreagan',
 'rules of engagement',
 'social security',
 'socialism',
 'supremecourt',
 'syria',
 'torture',
 'trump v/s bush',
 'trumps opinion change',
 'veterans',
 'wall street',
 "women's issues (not abortion though)",
 'zikavirus']

In [102]:
# remove intro closing nan and flipflopping 

T_new =T_new[~T_new['topic'].isin(['intro','flipflopping','closing','trump v/s bush','trumps opinion change','billclinton'])]
T_new.shape

(5581, 2)

In [103]:
# set all nans in topic to none
T_new = T_new[~T_new['topic'].isnull()]

In [104]:
T_new['topic'].value_counts()

none                                    1321
religion                                 407
women's issues (not abortion though)     371
racial issues                            365
abortion                                 293
jobs and economy                         268
immigration                              238
cnn                                      228
fox news or moderators                   188
healthcare                               140
socialism                                137
iran                                     136
lgbt issues                              126
gun control                              103
flintwater                               102
isis                                      98
israel                                    92
syria                                     86
benghazi                                  85
supremecourt                              81
poverty                                   75
foreign policy                            74
veterans  

In [105]:
# Sample the data so as to get equal numbers of each class.
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(T_new['ascii'],T_new['topic'], test_size=0.3, random_state=42)


In [106]:
    
print len(X_train), len(X_test) , len(y_train) , len(y_test)

3906 1675 3906 1675


In [107]:
stop_words = pd.read_table('/Users/preetijavaji/Documents/DSE/capstone/stop_words.org',names='words')
stop_words_list = list(stop_words['w'])
from sklearn.feature_extraction.text import CountVectorizer
vectorizer   =  CountVectorizer(min_df = 1,stop_words = stop_words_list, ngram_range = (1,2))
X_f_train = vectorizer.fit_transform(X_train)

In [108]:
classifiers = []
#classifiers.append( svm.SVC() )
classifiers.append( OneVsOneClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)) )

#classifier_1 = OneVsOneClassifier(svm.SVC())
#OneVsOneClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

cv = 10  # Number of crossvalidation folds to use.
scores = []

for classifier in classifiers:
    scores.append( cross_validation.cross_val_score(classifier, X_f_train, y_train, cv=cv) )
    
for score in scores:
    print score
    print score.mean()
    print

print 'done'

[ 0.80048662  0.80987654  0.80645161  0.82070707  0.79389313  0.81395349
  0.8203125   0.82849604  0.83244681  0.83870968]
0.816533349111

[ 0.82725061  0.82469136  0.84119107  0.84343434  0.81679389  0.81912145
  0.84114583  0.84960422  0.85904255  0.84946237]
0.837173769064

[ 0.77615572  0.77777778  0.77667494  0.8030303   0.7735369   0.78294574
  0.79947917  0.80474934  0.80585106  0.81989247]
0.792009341263

done


In [109]:
XT_f = vectorizer.transform(X_test)
predicteds = []
for classifier in classifiers:
    classifier.fit(X_f_train, y_train)
    yy = classifier.predict(XT_f)
    print yy.shape
    predicteds.append( classifier.predict(XT_f))


(1675,)
(1675,)
(1675,)


In [110]:
# Apply classifiers to the whole (not sampled) dataset and look at the confusion matrix and classification report for each.

# Format the whole dataset.
X_all = T_new['ascii'].values
y_all = T_new['topic'].values


# Create feature vectors from the tweets. Use the same feature vector as above.
X_featurized_all = vectorizer.transform(X_all)

# Generate predictions using the classifier trained on the equally sampled data.
predicteds = []
for classifier in classifiers:
    predicteds.append( classifier.predict(X_featurized_all) )
    
for y_pred_all in predicteds:
    print accuracy_score(y_all, y_pred_all)
    #print '   Neg ', 'Neu ', 'Pos'
    print confusion_matrix(y_all, y_pred_all)
    print classification_report(y_all, y_pred_all)
    print


0.937645583229
[[249   0   0 ...,   0  19   0]
 [  0  83   0 ...,   0   0   0]
 [  0   0   5 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   3   0   0]
 [  6   0   0 ...,   0 338   0]
 [  0   0   0 ...,   0   0  31]]
             precision    recall  f1-score   support

   abortion       0.90      0.85      0.88       293
   benghazi       0.98      0.98      0.98        85
campaign finance       1.00      1.00      1.00         5
clinton emails       0.97      0.91      0.94        35
        cnn       0.97      0.97      0.97       228
    college       1.00      1.00      1.00        22
common core       1.00      1.00      1.00         1
conservatism       1.00      1.00      1.00        23
deathpenalty       1.00      1.00      1.00        35
environment       1.00      1.00      1.00        32
family leave       1.00      0.96      0.98        23
 flintwater       1.00      0.98      0.99       102
foreign policy       0.97      0.84      0.90        74
fox news or moderators      

In [111]:
# read the tweet sample into a df
Tweet_df = pd.read_table('Tweet_g4.csv',sep=',',names=['tweetid','tweet','c3','c4','c5','c6','c7','c8'])
Tweet_df

,tweetid,tweet,c3,c4,c5,c6,c7,c8
0,NaN,"""BERNIE: all people deserve to be treated equally HILLARY: the oposite of ""bernie sanders"" is ""soakey waterers"" BERNIE: … HILLARY: pls fav/rt """,Tweet_2016_2_9,1454975357000,NaN,696843271293833216,Mon Feb 08 23:49:17 +0000 2016,492
1,6.968433e+17,"""RT @jonnysun: BERNIE: all people deserve to be treated equally HILLARY: the oposite of ""bernie sanders"" is ""soakey waterers"" BERNIE: … HILL… """,Tweet_2016_2_9,1454976000066,NaN,696845967438512128,Tue Feb 09 00:00:00 +0000 2016,0
2,NaN,"""Trump strode onstage yesterday to ""Revolution"" by the Beatles. https://t.co/ewPnuse1oT """,Tweet_2016_2_9,1454975569000,NaN,696844162944135168,Mon Feb 08 23:52:49 +0000 2016,0
3,NaN,"""How many times is going to be sued by rock bands? https://t.co/JWM1UXM4zY """,Tweet_2016_2_9,1454976000106,NaN,696845967606181889,Tue Feb 09 00:00:00 +0000 2016,0
4,NaN,"""How Lady Gaga Saved The Pantsuit: Hillary Clinton makes gentle fun of herself wearing them, and the pantsuit h... https://t.co/snMWYc8hLj """,Tweet_2016_2_9,1454976000226,NaN,696845968109543424,Tue Feb 09 00:00:00 +0000 2016,0
5,NaN,"""@RobL777 @FoxNews None will win general. Rubio &amp; maybe Trump. No Bush, Kasich, . Christie stole NJ pensions,not cleared from #Bridgegate. """,Tweet_2016_2_9,1454976000190,6.968398e+17,696845967958667264,Tue Feb 09 00:00:00 +0000 2016,0
6,NaN,"""What kind of small-minded, insecure POS coward creates a website just to gather Trump supporters together to deliberately suspend accounts? """,Tweet_2016_2_9,1454957906000,NaN,696770076242907138,Mon Feb 08 18:58:26 +0000 2016,127
7,6.967701e+17,"""RT @PolitiBunny: What kind of small-minded, insecure POS coward creates a website just to gather Trump supporters together to deliberately … """,Tweet_2016_2_9,1454976000162,NaN,696845967841222657,Tue Feb 09 00:00:00 +0000 2016,0
8,NaN,"""So let’s all be for @tedcruz and then we won’t have to worry about all this infighting :-))))))))))) Sorry for the column’s worth of tweets """,Tweet_2016_2_9,1454966439000,NaN,696805866054389760,Mon Feb 08 21:20:39 +0000 2016,72
9,6.968059e+17,"""RT @DavidLimbaugh: So let’s all be for @tedcruz and then we won’t have to worry about all this infighting :-))))))))))) Sorry for the colu… """,Tweet_2016_2_9,1454976000278,NaN,696845968327602176,Tue Feb 09 00:00:00 +0000 2016,0


In [112]:
# copy classifier 0 as the model for g1 and test it in spark on a sample 
# Convert to ascii
Tweet_df['ascii'] = Tweet_df['tweet'].apply(to_ascii)

# Remove hashtags
#df['ascii'] = df['ascii'].str.replace(r"#([A-Za-z0-9_]+)", " ")

# Remove handles
#df['ascii'] = df['ascii'].str.replace(r"@([A-Za-z0-9_]+)", " ")

# Remove URLs
#df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", " ")
# Replace URLs with "URL"
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"(['';:@%#()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"([;:@%#()\+\*\"\…\“\”])", "")

# Remove eol symbols
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbolst
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
Tweet_df['ascii'] = Tweet_df['ascii'].str.lower()


Tweet_df[['tweet', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)


,tweet,ascii
0,"""BERNIE: all people deserve to be treated equally HILLARY: the oposite of ""bernie sanders"" is ""soakey waterers"" BERNIE: … HILLARY: pls fav/rt """,bernie all people deserve to be treated equally hillary the oposite of bernie sanders is soakey waterers bernie hillary pls fav/rt
1,"""RT @jonnysun: BERNIE: all people deserve to be treated equally HILLARY: the oposite of ""bernie sanders"" is ""soakey waterers"" BERNIE: … HILL… """,rt @jonnysun bernie all people deserve to be treated equally hillary the oposite of bernie sanders is soakey waterers bernie hill
2,"""Trump strode onstage yesterday to ""Revolution"" by the Beatles. https://t.co/ewPnuse1oT """,trump strode onstage yesterday to revolution by the beatles. url


In [113]:
X = Tweet_df['ascii'].values

#train_vectors = vectorizer.fit_transform(train_data)
#test_vectors = vectorizer.transform(test_data)
X_featurized = vectorizer.transform(X)
        
print X_featurized.shape
print 'done'

(10000, 35962)
done


In [114]:
predicted_new = []
for classifier in classifiers:
    predicted_new.append( classifier.predict(X_featurized) )

In [115]:
Tweet_df['topic'] = predicted_new[0]

In [116]:
from collections import Counter

Keys = Counter(predicted_new[0]).keys() # equals to list(set(words))
values =  Counter(predicted_new[0]).values() # counts the elements' frequency
for k,v in zip(Keys,values):
    print k,':',v

racial issues : 325
president bush : 1
fox news or moderators : 44
marijuana : 3
college : 7
family leave : 13
lgbt issues : 41
environment : 15
religion : 43
clinton emails : 33
cnn : 224
veterans : 31
social security : 14
benghazi : 41
israel : 31
iran : 8
torture : 3
flintwater : 29
immigration : 166
poverty : 5
foreign policy : 39
iraq : 13
obama : 195
none : 5886
isis : 18
gun control : 15
jobs and economy : 874
abortion : 98
zikavirus : 2
socialism : 1327
conservatism : 12
healthcare : 82
syria : 11
women's issues (not abortion though) : 351


In [122]:
Tweet_df[Tweet_df['topic'] == "clinton emails"]['tweet']

110                                                     "BREAKING: FBI confirms investigation into @HillaryClinton's emails.  https://t.co/Ukas4izrl0 "
325                                       "Hillary Clinton emails: FBI probe confirmed by State Department - Washington Times https://t.co/JXeQeF6hxR "
326                       "RT @dvernychuk: Hillary Clinton emails: FBI probe confirmed by State Department - Washington Times https://t.co/JXeQeF6hxR "
698              "FBI (@FBI): Hillary Clinton Email Probe 'Ongoing'. https://t.co/vEt7dQOESu SRC @politico #Clinton #HillaryClinton (@HillaryClinton) "
1415                                                "FBI investigation into Clinton emails made official on eve of NH primary https://t.co/wAVkNTPeM2 "
2149       "Bill Clinton rips ‘sexist’ attacks: Bill Clinton lashed out at "sexist" and "profane" attack... https://t.co/uShUB2UWS9 #InformationVilla "
2433                                          "FBI confirms investigation into Hillary C

In [120]:
# copy classifier 0 as the model for g1 and test it in spark on a sample 
import pickle
# save model in a pickle file
output = open('mymodel_g4.pkl', 'wb')
pickle.dump(classifiers[0],output)
output.close()

In [121]:
vec_file = open('myvec_g4.pkl','wb')
pickle.dump(vectorizer,vec_file)
vec_file.close()